https://github.com/BAMresearch/NFDI4IngScientificWorkflowRequirements

In [1]:
import os

In [2]:
import subprocess

In [3]:
from pyiron_base import Project

In [4]:
def create_env(env_key):
    subprocess.check_output(
        ["mamba", "env", "create", "-n", env_key, "-f", env_dict[env_key], "-y"],
        universal_newlines=True,
    ).split("\n")

In [5]:
env_dict = {
    "preprocessing": "source/envs/preprocessing.yaml",
    "processing": "source/envs/processing.yaml",
    "postprocessing": "source/envs/postprocessing.yaml"
}

In [6]:
paper_source = "source/paper.tex"
macros_source = "source/macros.tex.template"

In [7]:
pr = Project(".")

# Preprocessing 

In [8]:
env_key = "preprocessing"

In [9]:
create_env(env_key=env_key)

In [10]:
pr_pre = Project("preprocessing")

In [11]:
pr_pre.remove_jobs(recursive=True, silently=True)

  0%|          | 0/1 [00:00<?, ?it/s]

## generate mesh

In [12]:
def write_input(input_dict, working_directory):
    script_name = os.path.join(working_directory, "gmsh.sh")
    with open(script_name, "w") as f:
        f.writelines("gmsh -2 -setnumber domain_size " + str(input_dict["domain_size"]) + " unit_square.geo -o square.msh")
    os.chmod(script_name, 0o744)

In [13]:
def collect_output(working_directory):
    return {}

In [14]:
pr.create_job_class(
    class_name="GmshJob",
    write_input_funct=write_input,
    collect_output_funct=collect_output,
    default_input_dict={"domain_size": 2.0},
    executable_str="./gmsh.sh",
)

In [15]:
job_gmsh = pr_pre.create.job.GmshJob(job_name="gmsh")

In [16]:
job_gmsh.server.conda_environment_name = env_key

In [17]:
job_gmsh.restart_file_list.append("source/unit_square.geo")

In [18]:
job_gmsh.run()

The job gmsh was saved and received the ID: 1


EnvironmentLocationNotFound: Not a conda environment: /srv/conda/envs/notebook/envs/preprocessing

In [ ]:
domain_size = 2.0

In [ ]:
gmsh_input = "source/unit_square.geo"
gmsh_output = "preprocessing/square.msh"

In [ ]:
subprocess.check_output(
    ["conda", "run", "-n", env_key, "gmsh", "-2", "-setnumber", "domain_size", str(domain_size), gmsh_input, "-o", gmsh_output],
    universal_newlines=True,
).split("\n")

## convert to xdmf

In [ ]:
meshio_output = "preprocessing/square.xdmf"

In [ ]:
subprocess.check_output(
    ["conda", "run", "-n", env_key, "meshio", "convert", gmsh_output, meshio_output],
    universal_newlines=True,
).split("\n")

# Processing

In [ ]:
env_key = "processing"

In [ ]:
create_env(env_key=env_key)

In [ ]:
os.makedirs("processing", exist_ok=True)

## poisson

In [ ]:
poisson_output_pvd =  "processing/poisson.pvd"
poisson_output_numdofs = "processing/numdofs.txt"

In [ ]:
subprocess.check_output(
    ["conda", "run", "-n", env_key, "python", "source/poisson.py", "--mesh", meshio_output, "--degree", "2", "--outputfile", poisson_output_pvd, "--num-dofs", poisson_output_numdofs],
    universal_newlines=True,
).split("\n")

# Postprocessing

In [ ]:
env_key = "postprocessing"

In [ ]:
create_env(env_key=env_key)

## plot over line

In [ ]:
os.makedirs("postprocessing", exist_ok=True)

In [ ]:
pvbatch_output = "postprocessing/plotoverline.csv"

In [ ]:
subprocess.check_output(
    ["conda", "run", "-n", env_key, "pvbatch", "source/postprocessing.py", poisson_output_pvd, pvbatch_output],
    universal_newlines=True,
).split("\n")

## substitute macros

In [ ]:
with open(poisson_output_numdofs, "r") as instream:
    ndofs = int(instream.read())

In [ ]:
macros_output = "macros.tex"

In [ ]:
subprocess.check_output(
    ["conda", "run", "-n", env_key, "python", "source/prepare_paper_macros.py", "--macro-template-file", macros_source, "--plot-data-path", pvbatch_output, "--domain-size", str(domain_size), "--num-dofs", str(ndofs), "--output-macro-file", macros_output],
    universal_newlines=True,
).split("\n")

## compile paper

In [ ]:
paper_output = "paper.tex"

In [ ]:
subprocess.check_output(
    ["cp", paper_source, paper_output],
    universal_newlines=True,
).split("\n")

In [ ]:
subprocess.check_output(
    ["conda", "run", "-n", env_key, "tectonic", paper_output],
    universal_newlines=True,
).split("\n")

# Overview


In [ ]:
! mamba env list